In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import scanpy as sc

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    print("No GPU available. Training will run on CPU.")
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)





GPU: Tesla V100-PCIE-32GB is available.
cuda


In [2]:
GeneExp = sc.read_csv('GSE100866_PBMC_vs_flow_10X-RNA_umi.csv').transpose()
SurfaceProtein = sc.read_csv('GSE100866_PBMC_vs_flow_10X-ADT_umi.csv').transpose()

In [4]:
from sklearn.preprocessing import minmax_scale
GeneExp.var["mt"] = GeneExp.var_names.str.startswith("MOUSE_")
#Filtering out Mouse Mitocondrial Genes
GeneExpHuman = GeneExp[:,~GeneExp.var["mt"]]
GeneExpHumanNorm = GeneExp[:,~GeneExp.var["mt"]]
#Normalization 0 and 1
GeneExpHumanNorm.X = minmax_scale(GeneExpHumanNorm.X, feature_range=(0, 1), axis=0)
SurfaceProtein.X = minmax_scale(SurfaceProtein.X, feature_range=(0, 1), axis=0)



In [5]:
X_train, X_test, y_train, y_test = train_test_split(GeneExpHumanNorm.X, SurfaceProtein.X, test_size=0.2, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
#CNN MODEl
class CNNModel(nn.Module):
    def __init__(self, input_length):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=1)
        self.pool = nn.MaxPool1d(kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=1)
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=1)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * input_length, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool(x)
        x = torch.relu(self.conv2(x))
        x = self.pool(x)
        x = torch.relu(self.conv3(x))
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


input_length = X_train_tensor.shape[2]
model = CNNModel(input_length=input_length).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    model.eval()
    validation_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            validation_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {running_loss/len(train_loader)}, Validation Loss: {validation_loss/len(test_loader)}')
    
torch.save(model.state_dict(), '')

print("Training complete and model saved.")

Epoch 1/20, Training Loss: 1.7993671120004728, Validation Loss: 0.0088581214658916
Epoch 2/20, Training Loss: 0.008386044597718864, Validation Loss: 0.008389950152486563
Epoch 3/20, Training Loss: 0.008988560349680484, Validation Loss: 0.006123717748560012
Epoch 4/20, Training Loss: 0.004796816689195111, Validation Loss: 0.006232774006202817
Epoch 5/20, Training Loss: 0.0037295732233906167, Validation Loss: 0.005564334406517446
Epoch 6/20, Training Loss: 0.00313830058905296, Validation Loss: 0.0058624006062746046
Epoch 7/20, Training Loss: 0.0029117262281943114, Validation Loss: 0.00544658332131803
Epoch 8/20, Training Loss: 0.0019509366585407405, Validation Loss: 0.005333270751871169
Epoch 9/20, Training Loss: 0.001579398706671782, Validation Loss: 0.0054122464312240486
Epoch 10/20, Training Loss: 0.0013752725694212132, Validation Loss: 0.005409406200051307
Epoch 11/20, Training Loss: 0.001267264587804675, Validation Loss: 0.005533583625219762
Epoch 12/20, Training Loss: 0.00118661231

In [23]:
class CNNModel(nn.Module):
    def __init__(self, input_length):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=1)
        self.pool = nn.MaxPool1d(kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=1)
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=1)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * input_length, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool(x)
        x = torch.relu(self.conv2(x))
        x = self.pool(x)
        x = torch.relu(self.conv3(x))
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
model = CNNModel(X_test_tensor.shape[2])
model.load_state_dict(torch.load('pyToch_model.cnn',map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
model.eval()

y_pred = model(X_test_tensor)

In [26]:
y_pred

tensor([[ 0.2914,  0.3518, -0.0014,  ...,  0.0155,  0.0116,  0.0031],
        [-0.0359, -0.0188,  0.0190,  ...,  0.0187, -0.0047,  0.1139],
        [ 0.0827,  0.0093,  0.1911,  ...,  0.0088,  0.0158, -0.0054],
        ...,
        [ 0.1269,  0.0365,  0.2774,  ...,  0.0175, -0.0127,  0.0183],
        [ 0.3086,  0.3159,  0.0773,  ...,  0.0199,  0.0237,  0.0046],
        [ 0.0556,  0.1082,  0.0238,  ...,  0.1467,  0.1390, -0.0310]],
       grad_fn=<AddmmBackward0>)

In [10]:
#FeedForward Network
class FeedForward(nn.Module):
    def __init__(self, input_length):
        super(FeedForward, self).__init__()
        self.fc1 = nn.Linear(input_length, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Assuming X_train_tensor is already defined and loaded
input_length = X_train_tensor.shape[1] * X_train_tensor.shape[2]  # Flatten the input dimensions
model = FeedForward(input_length=input_length).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        # Flatten the input batch to (batch_size, input_length)
        X_batch = X_batch.view(X_batch.size(0), -1)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    model.eval()
    validation_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            # Flatten the input batch to (batch_size, input_length)
            X_batch = X_batch.view(X_batch.size(0), -1)

            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            validation_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {running_loss/len(train_loader)}, Validation Loss: {validation_loss/len(test_loader)}')

# Specify the model saving path
torch.save(model.state_dict(), 'feedforward_model.pth')

print("Training complete and model saved.")


Epoch 1/20, Training Loss: 0.0059963228669948875, Validation Loss: 0.004267849405296147
Epoch 2/20, Training Loss: 0.0023498309176648038, Validation Loss: 0.004231323562562466
Epoch 3/20, Training Loss: 0.0011388072413683404, Validation Loss: 0.004346407670527697
Epoch 4/20, Training Loss: 0.0006652786447375547, Validation Loss: 0.004586421204730868
Epoch 5/20, Training Loss: 0.0006330406895722262, Validation Loss: 0.004345176634378731
Epoch 6/20, Training Loss: 0.0005518805733299814, Validation Loss: 0.004299844042398035
Epoch 7/20, Training Loss: 0.00047939546246198003, Validation Loss: 0.0043811866175383326
Epoch 8/20, Training Loss: 0.0003932729602092877, Validation Loss: 0.004289861763827503
Epoch 9/20, Training Loss: 0.0002983661885809852, Validation Loss: 0.004328508526086807
Epoch 10/20, Training Loss: 0.0002479947999745491, Validation Loss: 0.004298337497748435
Epoch 11/20, Training Loss: 0.00022868338659463915, Validation Loss: 0.004254807597026229
Epoch 12/20, Training Loss:

In [12]:
class FeedForward(nn.Module):
    def __init__(self, input_length):
        super(FeedForward, self).__init__()
        self.fc1 = nn.Linear(input_length, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = FeedForward(X_test_tensor.shape[2])
model.load_state_dict(torch.load('feedforward_model.pth',map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
model.eval()

y_pred = model(X_test_tensor)

In [13]:
y_pred # FF_output

tensor([[[ 0.3367,  0.3662, -0.0190,  ...,  0.0150,  0.0135, -0.0021]],

        [[ 0.0114,  0.0065, -0.0047,  ...,  0.0093,  0.0149,  0.0643]],

        [[ 0.1030,  0.0091,  0.1599,  ...,  0.0145,  0.0064,  0.0054]],

        ...,

        [[ 0.1303,  0.0265,  0.2505,  ...,  0.0192,  0.0087, -0.0028]],

        [[ 0.2292,  0.3446,  0.0068,  ...,  0.0283,  0.0108,  0.0016]],

        [[ 0.0323,  0.0752,  0.0078,  ...,  0.1811,  0.1622, -0.0044]]],
       grad_fn=<ViewBackward0>)